In [7]:
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import silhouette_score

import pandas as pd
from datetime import datetime
import time

In [16]:
data  = pd.read_csv('../Data/earthquake_dataset.csv')
data.head()

,Time,Latitude,Longitude,Depth/Km,Magnitude,EventLocationName
0,2010-10-13T21:41:46.570000,42.623,12.756,10.5,1.7,3 km W Ferentillo (TR)
1,2010-10-13T21:43:14.530000,42.457,13.390,10.8,1.7,8 km E Pizzoli (AQ)
2,2010-10-13T23:35:35.700000,42.470,13.377,11.0,0.8,8 km E Pizzoli (AQ)
3,2010-10-13T23:44:28.160000,42.474,13.393,12.9,1.3,9 km E Pizzoli (AQ)
4,2010-10-13T23:46:11.610000,42.448,13.387,10.2,2.0,8 km E Pizzoli (AQ)


In [17]:
dates = [datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f') for x in data['Time']]
time_s = [time.mktime(d.timetuple()) / (60*60*24) for d in dates]  # days since epoch

data['Time/s'] = time_s

# select features
features = ['Latitude', 'Longitude', 'Depth/Km', 'Magnitude', 'Time/s'] # 'Year', 'Month', 'Day']#, 'Hour']
data = data[features]

data.head()

,Latitude,Longitude,Depth/Km,Magnitude,Time/s
0,42.623,12.756,10.5,1.7,14895.820671
1,42.457,13.390,10.8,1.7,14895.821690
2,42.470,13.377,11.0,0.8,14895.899711
3,42.474,13.393,12.9,1.3,14895.905880
4,42.448,13.387,10.2,2.0,14895.907072


In [18]:
X = data.values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X.shape

(29969, 5)

# K-means

In [19]:
# choice of best value of  K that minimizes the sum of squared error
sse_list = list([0] * 49)
silhouette_list = list([0] * 49)
max_k = 10

for k in range(2, max_k + 1):
    kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10, max_iter=100)
    kmeans.fit(X)
    sse_list[k-2] = kmeans.inertia_
    silhouette_list[k-2] = silhouette_score(X, kmeans.labels_)

MemoryError: 

In [ ]:
# plot SSE and silhouette on the same scale
# first axis
fig, ax1 = plt.subplots()
ax1.plot(range(2, max_k+1), sse_list, 'b')
ax1.set_xlabel('K')
ax1.set_ylabel('SSE', color='b')
ax1.tick_params('y', colors='b')
# second axis
ax2 = ax1.twinx()
ax2.plot(range(2, max_k+1), silhouette_list, 'orange')
ax2.set_ylabel('Silhouette score', color='orange')
ax2.tick_params('y', colors='orange')

plt.title('K-Means: SSE and Silhouette score vs K')
fig.tight_layout()
plt.savefig('../images/sse_silhouette_vs_k.png')
plt.show()